# Spark SQL & ML Mini-Project
This notebook contains answers and explorations for the Databricks mini-project using the `adult` dataset.

## Exercise Overview
We'll use Spark SQL, DataFrames, and build simple classification models. Dataset: `/databricks-datasets/adult/adult.data`.

## Step 1: Load & Register Data

In [1]:
# Load and register the adult dataset as a Spark SQL table
spark.sql("""DROP TABLE IF EXISTS adult""")
spark.sql("""
CREATE TABLE adult (
  age DOUBLE,
  workclass STRING,
  fnlwgt DOUBLE,
  education STRING,
  education_num DOUBLE,
  marital_status STRING,
  occupation STRING,
  relationship STRING,
  race STRING,
  sex STRING,
  capital_gain DOUBLE,
  capital_loss DOUBLE,
  hours_per_week DOUBLE,
  native_country STRING,
  income STRING)
USING com.databricks.spark.csv
OPTIONS (path "/databricks-datasets/adult/adult.data", header "true")
""")

NameError: name 'spark' is not defined

## Question 1: Top `bachelor_rate` by education (Spark SQL)

In [ ]:
result = spark.sql("""
SELECT
  education,
  SUM(1) as n,
  ROUND(AVG(if(LTRIM(marital_status) = 'Never-married',1,0)),2) as bachelor_rate
FROM adult 
GROUP BY 1
ORDER BY bachelor_rate DESC
""")
result.show(1)

## Question 2: Top `bachelor_rate` by education (DataFrame API)

In [ ]:
from pyspark.sql.functions import when, col, mean, desc, round

df_adult = spark.table("adult")

df_result = df_adult.select(
    df_adult['education'],
    when(col('marital_status') == ' Never-married', 1).otherwise(0).alias('is_bachelor')
)

df_result = df_result.groupBy('education').agg(round(mean('is_bachelor'), 2).alias('bachelor_rate'))
df_result = df_result.orderBy(desc('bachelor_rate'))

df_result.show(1)

## Income Distribution by Occupation

In [ ]:
import pandas as pd

result = spark.sql("""
SELECT 
  occupation,
  AVG(IF(income = ' >50K',1,0)) as plus_50k
FROM adult 
GROUP BY 1
ORDER BY 2 DESC
""")

df_pandas = pd.DataFrame(result.collect(), columns=result.schema.names)
df_pandas.head()

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')
df_pandas.plot(kind='barh', x='occupation', y='plus_50k', figsize=(10, 6))
plt.title('Income >50K by Occupation')
plt.xlabel('Proportion >50K')
plt.ylabel('Occupation')
plt.tight_layout()
plt.show()